In [1]:
import pandas as pd
import json
import pprint
from datetime import datetime as dt
from pandas import DataFrame
import time
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

In [2]:
bus_df = pd.read_json('yelp_dataset/yelp_academic_dataset_business.json', lines=True)
pprint.pprint(bus_df)

                   business_id                     name               address  \
0       6iYb2HFDywm3zjuRg0shjw      Oskar Blues Taproom          921 Pearl St   
1       tCbdrRPZA0oiIYSmHG3J0w  Flying Elephants at PDX   7000 NE Airport Way   
2       bvN78flM8NLprQ1a1y5dRg           The Reclaimory    4720 Hawthorne Ave   
3       oaepsyvc0J17qwi8cfrOWg              Great Clips    2566 Enterprise Rd   
4       PE9uqAjdw0E4-8mjGl3wVA        Crossfit Terminus   1046 Memorial Dr SE   
...                        ...                      ...                   ...   
160580  D2mHoIDXx9N8mS1pGoKV9Q       Eleven Oaks Realty                         
160581  bQX-kwVTyZgcdZGEPzce6Q             Webb's Honey    21777 State Rd 520   
160582  wvFZ06nmPmQ2-IVoPqVYLA    Painting with a Twist  2164 S Chickasaw Trl   
160583  GB75wPibj3IjNauaoCxyGA              Havana Cafe        910 NW 14th St   
160584  ngmLL5Y5OT-bYHKU0kKrYA              Zora Grille   1370 E Altamonte Dr   

                     city s

In [3]:
bus_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ..."
2,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,45.511907,-122.613693,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Antiques, Fashion, Used, Vintage & Consignment...","{'Thursday': '11:0-18:0', 'Friday': '11:0-18:0..."
3,oaepsyvc0J17qwi8cfrOWg,Great Clips,2566 Enterprise Rd,Orange City,FL,32763,28.914482,-81.295979,3.0,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Beauty & Spas, Hair Salons",None
4,PE9uqAjdw0E4-8mjGl3wVA,Crossfit Terminus,1046 Memorial Dr SE,Atlanta,GA,30316,33.747027,-84.353424,4.0,14,1,"{'GoodForKids': 'False', 'BusinessParking': '{...","Gyms, Active Life, Interval Training Gyms, Fit...","{'Monday': '16:0-19:0', 'Tuesday': '16:0-19:0'..."


In [4]:
or_sorted = bus_df.loc[(bus_df['state'] == 'OR')]
or_sorted

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ..."
2,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,45.511907,-122.613693,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Antiques, Fashion, Used, Vintage & Consignment...","{'Thursday': '11:0-18:0', 'Friday': '11:0-18:0..."
20,DLbEQySMW3X7KvkybpEYkw,Jasmine Nail Spa,"4370 SE King Rd, Ste 125",Portland,OR,97222,45.447598,-122.617427,3.0,7,0,"{'RestaurantsPriceRange2': '1', 'ByAppointment...","Nail Salons, Beauty & Spas","{'Monday': '9:30-18:0', 'Tuesday': '9:30-18:0'..."
23,j68RDnvOaYEwGcY9cTEHpA,Pittock Mansion,3229 NW Pittock Dr,Portland,OR,97210,45.525259,-122.716227,4.5,406,1,"{'BusinessAcceptsCreditCards': 'True', 'GoodFo...","Tours, Museums, Architectural Tours, Venues & ...","{'Monday': '0:0-0:0', 'Thursday': '10:0-16:0',..."
31,_bZOUUc6Sr_W3BxbOUJacw,Prince of Peace,14175 NW Cornell Rd,Portland,OR,97229,45.530958,-122.823780,3.5,5,1,None,"Education, Preschools","{'Monday': '8:0-15:30', 'Tuesday': '8:0-15:30'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160561,jYgqSazE0gUyI7qq086Dzw,Chart House,5700 SW Terwilliger,Portland,OR,97239,45.483154,-122.682748,3.5,457,1,"{'BusinessAcceptsCreditCards': 'True', 'Outdoo...","Event Planning & Services, Steakhouses, Seafoo...","{'Tuesday': '16:0-21:0', 'Wednesday': '16:0-21..."
160562,cgNDiWCaSlqqxx1A6r65bA,Hop N Cork,17450 Lower Boones Ferry Rd,Lake Oswego,OR,97035,45.399154,-122.737600,4.0,137,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Nightlife, Wine Bars, Beer Bar, Bars, Restaura...","{'Monday': '0:0-0:0', 'Tuesday': '16:0-20:0', ..."
160570,det-KjTfYoG2JfCZG3v04w,Village Home Education Resource Center,5150 SW Watson Ave,Beaverton,OR,97005,45.482428,-122.806687,4.0,9,1,None,"Educational Services, Education",None
160573,Zl6SUy6x9jqjRu2HbtEO6A,Split Dine and Drink,7335 SW Bridgeport Rd,Tigard,OR,97224,45.394457,-122.752535,2.0,48,0,"{'RestaurantsDelivery': 'False', 'GoodForKids'...","American (Traditional), Desserts, American (Ne...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."


In [5]:
tip_df = pd.read_json('yelp_dataset/yelp_academic_dataset_tip.json', lines=True)
pprint.pprint(tip_df)

                        user_id             business_id  \
0        WCjg0jdHXMlwbqS9tZUx8Q  ENwBByjpoa5Gg7tKgxqwLg   
1        42-Z02y9bABShAGZhuSzrQ  jKO4Og6ucdX2-YCTKQVYjg   
2        5u7E3LYp_3eB8dLuUBazXQ  9Bto7mky640ocgezVKSfVg   
3        wDWoMG5N9oI4DJ-p7z8EBg  XWFjKtRGZ9khRGtGg2ZvaA   
4        JmuFlorjjRshHTKzTwNtgg  mkrx0VhSMU3p3uhyJGCoWA   
...                         ...                     ...   
1162114  OAf2rC_Btse6aRc9LCyApw  BYKjzEUv6RksSVm5yqGPDA   
1162115  bI5zqhhpU4SlW-5BN6M1uw  C2kDo2CFGP_0073dgMmZNw   
1162116  vDMl9JwvkRNRVt_rjpQdZg  4CxF8c3MB7VAdY8zFb2cZQ   
1162117  iCc7UVah7zE1UwYPP3FcTg  IEHoGw0V5Lbdf4TJU58K0Q   
1162118  e9x-ZsbUPTVX3Xv_L9vSaw  Mt-6e5pSAft1f3sA8tTOsQ   

                                                      text  \
0                                     Carne asada chips...   
1        Best happy hour from 3pm to 6pm! $1 off martin...   
2        Nice people, skilled staff, clean location - b...   
3        1/2-price bowling & the "Very" Old

In [6]:
clean_tip_df = tip_df.dropna(how='all')

dated_tip_df = pd.to_datetime(tip_df["date"])
sorted_tip_df = tip_df.sort_values(["date"], ascending= False)
sorted_tip_df.head()

,user_id,business_id,text,date,compliment_count
669130,q4YuIIWaekF_lrYFCG440A,I2OblwJG8_XzFxxoHU0vVQ,Surgeries are only completed on Thursdays only...,2021-01-28 14:20:15,0
820468,63UoTNVbHn35th-pgp6UKw,vQ0ihvqI0Xpz8TxBIKK_qg,Best donuts in town. Staff is super friendly.,2021-01-28 14:11:48,0
508765,Ouz4R8N4pDyYBqML6zb-ZA,7rEr-_j7NT2YZMFIi3KdpA,Temporarily closed today due to covid.,2021-01-28 14:01:20,0
656329,Q_fbXeB2HyRg6CmctrICKw,wegBPIo-9hnuEtAaY2xYvw,Try the chocolate cookie-ice cream sandwich. ...,2021-01-28 13:26:55,0
1148517,eseZ_gMjqBQMtyHyPpDMLA,t5EgwRXG6MZSihdqugOSKg,Great place,2021-01-28 11:57:22,0


In [7]:
relevant_tip_df = sorted_tip_df[sorted_tip_df['date'] > "2019-1-1"]
relevant_tip_df

,user_id,business_id,text,date,compliment_count
669130,q4YuIIWaekF_lrYFCG440A,I2OblwJG8_XzFxxoHU0vVQ,Surgeries are only completed on Thursdays only...,2021-01-28 14:20:15,0
820468,63UoTNVbHn35th-pgp6UKw,vQ0ihvqI0Xpz8TxBIKK_qg,Best donuts in town. Staff is super friendly.,2021-01-28 14:11:48,0
508765,Ouz4R8N4pDyYBqML6zb-ZA,7rEr-_j7NT2YZMFIi3KdpA,Temporarily closed today due to covid.,2021-01-28 14:01:20,0
656329,Q_fbXeB2HyRg6CmctrICKw,wegBPIo-9hnuEtAaY2xYvw,Try the chocolate cookie-ice cream sandwich. ...,2021-01-28 13:26:55,0
1148517,eseZ_gMjqBQMtyHyPpDMLA,t5EgwRXG6MZSihdqugOSKg,Great place,2021-01-28 11:57:22,0
...,...,...,...,...,...
431164,K-2-4KsCg_fEvc7xrS5yyA,viqkunBBI6QOIZehME-7gg,Great,2019-01-01 00:35:54,0
399487,TSHoVpWNd-mwjAw_kkSOfQ,zmRHDJq8qgWoVYobHL2ggg,Go buy an Android. This mall kiosk is a joke. ...,2019-01-01 00:31:17,0
435268,aBQZ_rREkyaQRpH9XjnjSA,tUhIIaYSjBuSbJzcf6CGIg,"Good stuff, always fresh and hot!",2019-01-01 00:27:50,0
952571,bMGIATKbYqPu8lkyLirvBw,bORoF6fOYu39jHgzNyiKiA,The bathroom doors aren't marked! The Japanese...,2019-01-01 00:15:42,0


In [8]:
bus_tip_merged = pd.merge(or_sorted, relevant_tip_df, how ='left')
bus_tip_merged

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,user_id,text,date,compliment_count
0,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ...",KZt0oX86xm9JeMjCu3PP-w,Mmmmmmmm so good,2019-06-25 18:21:12,0.0
1,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,45.511907,-122.613693,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Antiques, Fashion, Used, Vintage & Consignment...","{'Thursday': '11:0-18:0', 'Friday': '11:0-18:0...",NaN,NaN,NaT,NaN
2,DLbEQySMW3X7KvkybpEYkw,Jasmine Nail Spa,"4370 SE King Rd, Ste 125",Portland,OR,97222,45.447598,-122.617427,3.0,7,0,"{'RestaurantsPriceRange2': '1', 'ByAppointment...","Nail Salons, Beauty & Spas","{'Monday': '9:30-18:0', 'Tuesday': '9:30-18:0'...",NaN,NaN,NaT,NaN
3,j68RDnvOaYEwGcY9cTEHpA,Pittock Mansion,3229 NW Pittock Dr,Portland,OR,97210,45.525259,-122.716227,4.5,406,1,"{'BusinessAcceptsCreditCards': 'True', 'GoodFo...","Tours, Museums, Architectural Tours, Venues & ...","{'Monday': '0:0-0:0', 'Thursday': '10:0-16:0',...",nxNO7_6U2Nu90v0IdnEhIg,Mansion sits on 46 acres.\nView of Mt. Hood.\n...,2019-11-22 06:30:06,0.0
4,j68RDnvOaYEwGcY9cTEHpA,Pittock Mansion,3229 NW Pittock Dr,Portland,OR,97210,45.525259,-122.716227,4.5,406,1,"{'BusinessAcceptsCreditCards': 'True', 'GoodFo...","Tours, Museums, Architectural Tours, Venues & ...","{'Monday': '0:0-0:0', 'Thursday': '10:0-16:0',...",RlnqZVl2swWXm9KCQaHHSQ,The park towards the back of the Mansion (faci...,2019-10-27 12:58:04,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35260,jYgqSazE0gUyI7qq086Dzw,Chart House,5700 SW Terwilliger,Portland,OR,97239,45.483154,-122.682748,3.5,457,1,"{'BusinessAcceptsCreditCards': 'True', 'Outdoo...","Event Planning & Services, Steakhouses, Seafoo...","{'Tuesday': '16:0-21:0', 'Wednesday': '16:0-21...",YmH8DxMBRJXlt_gJ2ouI6w,They're still back\n\nAlright!!,2019-03-20 22:10:20,0.0
35261,cgNDiWCaSlqqxx1A6r65bA,Hop N Cork,17450 Lower Boones Ferry Rd,Lake Oswego,OR,97035,45.399154,-122.737600,4.0,137,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Nightlife, Wine Bars, Beer Bar, Bars, Restaura...","{'Monday': '0:0-0:0', 'Tuesday': '16:0-20:0', ...",NaN,NaN,NaT,NaN
35262,det-KjTfYoG2JfCZG3v04w,Village Home Education Resource Center,5150 SW Watson Ave,Beaverton,OR,97005,45.482428,-122.806687,4.0,9,1,None,"Educational Services, Education",None,NaN,NaN,NaT,NaN
35263,Zl6SUy6x9jqjRu2HbtEO6A,Split Dine and Drink,7335 SW Bridgeport Rd,Tigard,OR,97224,45.394457,-122.752535,2.0,48,0,"{'RestaurantsDelivery': 'False', 'GoodForKids'...","American (Traditional), Desserts, American (Ne...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",NaN,NaN,NaT,NaN


In [9]:
dropped_merged = bus_tip_merged.drop(['state', 'review_count', 'is_open', 'attributes', 'hours', 'user_id', 'text', 'compliment_count', 'date'], axis=1)
dropped_merged

,business_id,name,address,city,postal_code,latitude,longitude,stars,categories
0,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,97218,45.588906,-122.593331,4.0,"Salad, Soup, Sandwiches, Delis, Restaurants, C..."
1,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,97214,45.511907,-122.613693,4.5,"Antiques, Fashion, Used, Vintage & Consignment..."
2,DLbEQySMW3X7KvkybpEYkw,Jasmine Nail Spa,"4370 SE King Rd, Ste 125",Portland,97222,45.447598,-122.617427,3.0,"Nail Salons, Beauty & Spas"
3,j68RDnvOaYEwGcY9cTEHpA,Pittock Mansion,3229 NW Pittock Dr,Portland,97210,45.525259,-122.716227,4.5,"Tours, Museums, Architectural Tours, Venues & ..."
4,j68RDnvOaYEwGcY9cTEHpA,Pittock Mansion,3229 NW Pittock Dr,Portland,97210,45.525259,-122.716227,4.5,"Tours, Museums, Architectural Tours, Venues & ..."
...,...,...,...,...,...,...,...,...,...
35260,jYgqSazE0gUyI7qq086Dzw,Chart House,5700 SW Terwilliger,Portland,97239,45.483154,-122.682748,3.5,"Event Planning & Services, Steakhouses, Seafoo..."
35261,cgNDiWCaSlqqxx1A6r65bA,Hop N Cork,17450 Lower Boones Ferry Rd,Lake Oswego,97035,45.399154,-122.737600,4.0,"Nightlife, Wine Bars, Beer Bar, Bars, Restaura..."
35262,det-KjTfYoG2JfCZG3v04w,Village Home Education Resource Center,5150 SW Watson Ave,Beaverton,97005,45.482428,-122.806687,4.0,"Educational Services, Education"
35263,Zl6SUy6x9jqjRu2HbtEO6A,Split Dine and Drink,7335 SW Bridgeport Rd,Tigard,97224,45.394457,-122.752535,2.0,"American (Traditional), Desserts, American (Ne..."


In [10]:
for col in dropped_merged:
    print (dropped_merged[col].apply(type))

0        <class 'str'>
1        <class 'str'>
2        <class 'str'>
3        <class 'str'>
4        <class 'str'>
             ...      
35260    <class 'str'>
35261    <class 'str'>
35262    <class 'str'>
35263    <class 'str'>
35264    <class 'str'>
Name: business_id, Length: 35265, dtype: object
0        <class 'str'>
1        <class 'str'>
2        <class 'str'>
3        <class 'str'>
4        <class 'str'>
             ...      
35260    <class 'str'>
35261    <class 'str'>
35262    <class 'str'>
35263    <class 'str'>
35264    <class 'str'>
Name: name, Length: 35265, dtype: object
0        <class 'str'>
1        <class 'str'>
2        <class 'str'>
3        <class 'str'>
4        <class 'str'>
             ...      
35260    <class 'str'>
35261    <class 'str'>
35262    <class 'str'>
35263    <class 'str'>
35264    <class 'str'>
Name: address, Length: 35265, dtype: object
0        <class 'str'>
1        <class 'str'>
2        <class 'str'>
3        <class 'str'>
4        <class 

In [11]:
uniq_cities = dropped_merged["city"].value_counts()
uniq_cities

Portland               25439
Beaverton               3494
Tigard                  1400
Lake Oswego              976
Clackamas                615
                       ...  
Historic Milwaukie         1
beaverton                  1
Northwest Clackamas        1
Prontland                  1
HILLSBORO                  1
Name: city, Length: 78, dtype: int64

In [12]:
city_list = dropped_merged["city"].unique()
city_list

array(['Portland', 'Tigard', 'Happy Valley', 'Clackamas', 'Lake Oswego',
       'Milwaukie', 'Tualatin', 'Beaverton', 'Gladstone', 'Gresham',
       'Oregon City', 'Hillsboro', 'West Linn', 'Aloha', 'Damascus',
       'Oak Grove', 'Milwaukee', 'TIGARD', 'Sherwood', 'North Portland',
       'King City', 'Rockwood', 'NE Portland', 'PORTLAND', 'Lake Grove',
       'Silverton', 'Tigard (Portland)', 'Durham', 'portland', 'Hillsbor',
       'Porltand', 'Milwauke', 'Phoenix', 'Fairview', 'TIgard', 'OR',
       'Washington County', 'Sellwood', 'beaverton Or', 'Boring',
       'CLACKAMAS', 'Wilsonville', 'West linn', 'Beavercreek',
       'MILWAUKIE', 'Bend', 'Troutland', 'Vancouver', 'Marylhurst',
       'Portland-Gateway Plaza', 'Prontland', 'Mission Viejo',
       'Lake Oswego, OR', 'milwaukie', 'Portand', 'Tillamoo', 'Damascas',
       'Portland, OR', 'HILLSBORO', 'Medford', 'Salem', 'BEAVERTON',
       'St Johns', 'Rivergrove', 'Happy valley', 'Lake Owego',
       'Northwest Clackamas', 'H

In [13]:
cities = ['Portland', 'Beaverton', 'Tigard', 'Lake Oswego', 'Clackmas']
smaller_df = dropped_merged.loc[dropped_merged['city'].isin(cities)]
smaller_df

,business_id,name,address,city,postal_code,latitude,longitude,stars,categories
0,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,97218,45.588906,-122.593331,4.0,"Salad, Soup, Sandwiches, Delis, Restaurants, C..."
1,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,97214,45.511907,-122.613693,4.5,"Antiques, Fashion, Used, Vintage & Consignment..."
2,DLbEQySMW3X7KvkybpEYkw,Jasmine Nail Spa,"4370 SE King Rd, Ste 125",Portland,97222,45.447598,-122.617427,3.0,"Nail Salons, Beauty & Spas"
3,j68RDnvOaYEwGcY9cTEHpA,Pittock Mansion,3229 NW Pittock Dr,Portland,97210,45.525259,-122.716227,4.5,"Tours, Museums, Architectural Tours, Venues & ..."
4,j68RDnvOaYEwGcY9cTEHpA,Pittock Mansion,3229 NW Pittock Dr,Portland,97210,45.525259,-122.716227,4.5,"Tours, Museums, Architectural Tours, Venues & ..."
...,...,...,...,...,...,...,...,...,...
35260,jYgqSazE0gUyI7qq086Dzw,Chart House,5700 SW Terwilliger,Portland,97239,45.483154,-122.682748,3.5,"Event Planning & Services, Steakhouses, Seafoo..."
35261,cgNDiWCaSlqqxx1A6r65bA,Hop N Cork,17450 Lower Boones Ferry Rd,Lake Oswego,97035,45.399154,-122.737600,4.0,"Nightlife, Wine Bars, Beer Bar, Bars, Restaura..."
35262,det-KjTfYoG2JfCZG3v04w,Village Home Education Resource Center,5150 SW Watson Ave,Beaverton,97005,45.482428,-122.806687,4.0,"Educational Services, Education"
35263,Zl6SUy6x9jqjRu2HbtEO6A,Split Dine and Drink,7335 SW Bridgeport Rd,Tigard,97224,45.394457,-122.752535,2.0,"American (Traditional), Desserts, American (Ne..."


In [ ]:
tokenized_text=sent_tokenize([‘categories’])

In [ ]:
bus_tip_merged.to_json('yelp_dataset/yelp_merged_db.json', orient = 'columns')

In [ ]:
test_bus_df = pd.read_json('yelp_dataset/yelp_merged_db.json', lines=True)
test_bus_df

In [ ]:
checkin_df = pd.read_json('yelp_dataset/yelp_checkin_data.json', lines=True)
checkin_df

In [ ]:
checkin_df_renamed = checkin_df.rename(columns={'date':'checkin_date'})
checkin_df_renamed

In [ ]:
(checkin_df_renamed.set_index('business_id')['checkin_date']
   .str.split(',')
   .explode()
   .rename('checkin_date')
   .reset_index())

In [ ]:
bus_tip_merged.dtypes

In [ ]:
checkin_df_renamed.dtypes